In [11]:
import requests
import json
import pandas as pd
import random
import time
from datetime import datetime
from timeit import default_timer as timer
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS, ALL_REGIONS

In [3]:
#Better readability for JSON output
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent = 4)
    print(text)

In [12]:
def get_trades(tid_start, tid_end, currency):
    if tid_start >= tid_end: #make sure valid starting and ending values provided
        tid = tid_start #initialize 'tid' to starting value
        tmp_list = [] #temp list to hold dfs from loop
        trades = pd.DataFrame() #final df of trades
        i = 0 #while-loop counter for sleep function
        
        gateway = ApiGateway("https://localbitcoins.com", regions = ["us-west-2"]) #Use ApiGateway from requests_ip_rotator to randomize IP used for requests to avoid rate limiting
        gateway.start() #open gateway, always shutdown as well

        session = requests.Session() #begin session
        session.mount("https://localbitcoins.com", gateway)

        
        try:
            while (tid >= tid_end):
                
                parameters = {
                    'max_tid': tid #passes current value of 'tid' as a parameter into the API request
                }

                url = 'https://localbitcoins.com/bitcoincharts/' + str(currency) + '/trades.json'

                t = session.get(url, params = parameters) #pull last 500 trades, starting with max_tid
                t.raise_for_status() #Returns HTTP error object
                df = pd.DataFrame(t.json()) #get JSON output into pd DataFrame
                tmp_list.append(df.iloc[:-1]) #append to temp list, omits last row to avoid repeat obs
                
                tid_last = df['tid'].iloc[-1] #gets the last tid of the previous dataset, new starting point for next loop
                tid = tid_last
                time.sleep(1 + (random.randint(0, 1000) / 1000))#sleep function to avoid API errors, this one moderates time between API requests
                i += 1 #loop counter
                
                if i%500 == 0: #only for multiples of 500 (get API error around 800 without IP randomization)
                    # time.sleep(1200) #sleep for 20 minutes
                    print('congrats! you have made ' + str(i) + ' API request, you insane person.')
                
                
                
        except Exception as e:
            print(t.status_code)
            print(e.response.text)
            print(tid)
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades
        
        else:
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades

    else:
        print('Please enter a value for tid_start that is greater than tid_end')

In [13]:
currencies = ['USD']
start_tid = 11382842
end_tid = 6000000


for currency in currencies:
    print(currency)
    start = timer()
    
    df = get_trades(start_tid, end_tid, currency)
    path = '../temporary/trades_' + str(currency) + '.csv'
    end = timer()
    time = end - start
    print(time)
    
    df.to_csv(path, index = False)

USD
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.
4364.1260477


In [14]:
df

,date,tid,price,amount,Currency
0,1514143600,11382842,22999.93,0.00108696,USD
1,1514143355,11382840,22999.93,0.00108696,USD
2,1514142925,11382832,23664.03,0.00190162,USD
3,1514143060,11382827,25701.66,0.00233448,USD
4,1514143006,11382825,80817.22,0.00030934,USD
...,...,...,...,...,...
1168154,1487818576,5999301,1520.83,0.24000000,USD
1168155,1487818250,5999295,1156.07,0.08650000,USD
1168156,1487818257,5999294,2783.96,0.17960000,USD
1168157,1487818543,5999292,1515.15,0.01320000,USD


In [15]:
datetime.fromtimestamp(1487818710)

datetime.datetime(2017, 2, 22, 19, 58, 30)

In [6]:
#See most recent trades
t = requests.get('https://localbitcoins.com/bitcoincharts/USD/trades.json')
jprint(t.json())

[
    {
        "amount": "0.00185264",
        "date": 1663861942,
        "price": "21590.81",
        "tid": 56487802
    },
    {
        "amount": "0.00962980",
        "date": 1663861988,
        "price": "18733.51",
        "tid": 56487776
    },
    {
        "amount": "0.00519657",
        "date": 1663861677,
        "price": "22707.29",
        "tid": 56487743
    },
    {
        "amount": "0.00265854",
        "date": 1663861991,
        "price": "22568.78",
        "tid": 56487739
    },
    {
        "amount": "0.00296460",
        "date": 1663862029,
        "price": "20238.82",
        "tid": 56487734
    },
    {
        "amount": "0.09501188",
        "date": 1663861810,
        "price": "21050.00",
        "tid": 56487728
    },
    {
        "amount": "0.00275869",
        "date": 1663861065,
        "price": "21299.96",
        "tid": 56487704
    },
    {
        "amount": "0.04436052",
        "date": 1663860778,
        "price": "19679.66",
        "tid": 564876

In [6]:
#Playing around with ip rotator

gateway = ApiGateway("https://localbitcoins.com", regions = ["us-west-2"])
gateway.start()

session = requests.Session()
session.mount("https://localbitcoins.com", gateway)

response = session.get("https://localbitcoins.com/bitcoincharts/USD/trades.json")
print(response.status_code)

gateway.shutdown()

Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
200
Deleting gateway for site 'https://localbitcoins.com'.
Deleted 1 endpoints with for site 'https://localbitcoins.com'.


['c2wrgvdb2j']

In [8]:
print(response.json())

[{'date': 1664315282, 'tid': 56509992, 'price': '19567.09', 'amount': '0.05110622'}, {'date': 1664315243, 'tid': 56509953, 'price': '19622.46', 'amount': '0.00050962'}, {'date': 1664313770, 'tid': 56509887, 'price': '18299.34', 'amount': '0.00304929'}, {'date': 1664313609, 'tid': 56509864, 'price': '21915.72', 'amount': '0.00912587'}, {'date': 1664313577, 'tid': 56509851, 'price': '21899.96', 'amount': '0.00251142'}, {'date': 1664313415, 'tid': 56509849, 'price': '19633.70', 'amount': '0.05093283'}, {'date': 1664313789, 'tid': 56509827, 'price': '21750.77', 'amount': '0.00367803'}, {'date': 1664314119, 'tid': 56509825, 'price': '18426.39', 'amount': '0.00884872'}, {'date': 1664314079, 'tid': 56509819, 'price': '19640.96', 'amount': '0.00050914'}, {'date': 1664312793, 'tid': 56509799, 'price': '19661.16', 'amount': '0.09830905'}, {'date': 1664312792, 'tid': 56509792, 'price': '20895.92', 'amount': '0.04307061'}, {'date': 1664312505, 'tid': 56509785, 'price': '21750.69', 'amount': '0.000